# 2. Fight arrival data for some airports are problematic, showing unusually high loads at 00:00. This notebook aims to fix this issue

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import datetime
from datetime import time
import get_GSE_tasks as TASK
import matplotlib.pyplot as plt

In [ ]:
def fix_time_format(date_str): # a function to fix "24:00" issue
    if "24:00" in date_str:
        date_part = date_str.split()[0]  # Extract date
        new_date = datetime.strptime(date_part, "%m/%d/%Y") + timedelta(days=1)  # Add a day
        return new_date.strftime("%m/%d/%Y 00:00")  # Format correctly
    return date_str

In [ ]:
# Show data issue with one example

import matplotlib.pyplot as plt
import numpy as np
file_path = 'all_flight_arrival_data/DFW_all_flight_arrival_data.csv'
all_flight_data = pd.read_csv(file_path)
all_flight_data['Arrival_time'] = all_flight_data['Date (MM/DD/YYYY)'] + ' ' + all_flight_data['Actual Arrival Time']
all_flight_data['Arrival_time'] = all_flight_data['Arrival_time'].apply(fix_time_format)
all_flight_data['Arrival_time'] = pd.to_datetime(all_flight_data['Arrival_time'])
all_flight_data['aircraft_type'] = all_flight_data['Carrier Code'].apply(lambda x: 'wide' if x == 'III' else 'narrow')
all_flight_data['Hour'] = all_flight_data['Arrival_time'].dt.hour
all_flight_data['Day'] = all_flight_data['Arrival_time'].dt.day
all_flight_data['Month'] = all_flight_data['Arrival_time'].dt.month    

# Groupby month, day, and hour to check outlier data  
all_flight_data_group = all_flight_data.groupby(['Month','Day','Hour']).agg({'aircraft_type':'count'})
all_flight_data_group = all_flight_data_group.reset_index()
all_flight_data_group = all_flight_data_group.sort_values(by = ['Month','Day','Hour'])
plt.plot(range(all_flight_data_group.shape[0]), all_flight_data_group.aircraft_type)  # Plot the chart
plt.show()  # display

In [ ]:
# All raw data
folder_path = Path('./all_flight_arrival_data')
all_files = folder_path.glob('*.csv')

In [ ]:
# Define a function to parallel the data clean and processing

def data_clean(file_path):
    all_flight_data = pd.read_csv(file_path)
    # Fix and parse datetime
    all_flight_data['Arrival_time'] = all_flight_data['Date (MM/DD/YYYY)'] + ' ' + all_flight_data['Actual Arrival Time']
    all_flight_data['Arrival_time'] = all_flight_data['Arrival_time'].apply(fix_time_format)
    all_flight_data['Arrival_time'] = pd.to_datetime(all_flight_data['Arrival_time'])
    
    # Label aircraft type
    all_flight_data['aircraft_type'] = all_flight_data['Carrier Code'].apply(lambda x: 'wide' if x == 'III' else 'narrow')
    
    # Extract hour, day, month
    all_flight_data['Hour'] = all_flight_data['Arrival_time'].dt.hour
    all_flight_data['Day'] = all_flight_data['Arrival_time'].dt.day
    all_flight_data['Month'] = all_flight_data['Arrival_time'].dt.month
    
    # Group by for hourly flight count
    all_flight_data_group = all_flight_data.groupby(['Month', 'Day', 'Hour']).agg({'aircraft_type': 'count'}).reset_index()
    all_flight_data_group = all_flight_data_group.sort_values(by=['Month', 'Day', 'Hour'])
    
    # Define threshold for outlier detection
    hourly_threshold = all_flight_data_group.aircraft_type.quantile(0.9) * 1.35
    
    # Identify outlier day(s)
    outlier_records = all_flight_data_group[(all_flight_data_group['aircraft_type'] > hourly_threshold)&(all_flight_data_group['Hour'] ==0)]
    
    if all_flight_data_group.aircraft_type.quantile(0.9) >= 3 and outlier_records.shape[0] > 0:
        outlier_days = outlier_records[['Month', 'Day']].drop_duplicates()
        
        # Normal days (not outliers)
        normal_days = all_flight_data_group[~all_flight_data_group[['Month', 'Day']].apply(tuple, axis=1).isin(outlier_days.apply(tuple, axis=1))]
        
        # Replace each outlier day with a randomly selected normal day
        for idx, row in outlier_days.iterrows():
            bad_month, bad_day = row.Month, row.Day
        
            # Filter original records for the bad day
            bad_day_data = all_flight_data[(all_flight_data['Month'] == bad_month) & (all_flight_data['Day'] == bad_day)]
        
            # Randomly sample a normal day
            normal_day = normal_days.sample(1).iloc[0]
            norm_month, norm_day = normal_day.Month, normal_day.Day
        
            # Get original normal day data
            normal_day_data = all_flight_data[(all_flight_data['Month'] == norm_month) & (all_flight_data['Day'] == norm_day)].copy()
        
            # Shift normal day datetime to match the outlier date
            delta_days = (pd.Timestamp(f'2023-{bad_month:02d}-{bad_day:02d}') - 
                          pd.Timestamp(f'2023-{norm_month:02d}-{norm_day:02d}')).days
            normal_day_data['Arrival_time'] = normal_day_data['Arrival_time'] + pd.Timedelta(days=delta_days)
        
            # Update date fields
            normal_day_data['Month'] = bad_month
            normal_day_data['Day'] = bad_day
            normal_day_data['Date (MM/DD/YYYY)'] = normal_day_data['Arrival_time'].dt.strftime('%m/%d/%Y')
            normal_day_data['Date (MM/DD/YYYY)'] = normal_day_data['Date (MM/DD/YYYY)'].astype(str)

        
            # Remove bad day and replace it
            all_flight_data = all_flight_data[~((all_flight_data['Month'] == bad_month) & (all_flight_data['Day'] == bad_day))]
            all_flight_data = pd.concat([all_flight_data, normal_day_data], ignore_index=True)
        all_flight_data.to_csv('./all_flight_arrival_data_New/' + str(file_path).split('/')[-1])    
    else:
        all_flight_data.to_csv('./all_flight_arrival_data_New/' + str(file_path).split('/')[-1]) 

In [ ]:
# Parallelizing using Pool.map()
import multiprocessing as mp

pool = mp.Pool(mp.cpu_count())
 
results = pool.map(data_clean, list(all_files))
 
pool.close()

In [ ]:
# (Manually) Check cleaned data one by one

folder_path = Path('./all_flight_arrival_data_New')
cnt = 1
cnt_outlier = 0
outlier_hour_list = []
for file_path in [i for i in list(folder_path.glob('*.csv')) if 'LAX' in str(i)]:
    all_flight_data = pd.read_csv(file_path)
    all_flight_data['Arrival_time'] = all_flight_data['Date (MM/DD/YYYY)'] + ' ' + all_flight_data['Actual Arrival Time']
    all_flight_data['Arrival_time'] = all_flight_data['Arrival_time'].apply(fix_time_format)
    all_flight_data['Arrival_time'] = pd.to_datetime(all_flight_data['Arrival_time'])
    all_flight_data['aircraft_type'] = all_flight_data['Carrier Code'].apply(lambda x: 'wide' if x == 'III' else 'narrow')
    all_flight_data['Hour'] = all_flight_data['Arrival_time'].dt.hour
    all_flight_data['Day'] = all_flight_data['Arrival_time'].dt.day
    all_flight_data['Month'] = all_flight_data['Arrival_time'].dt.month    
    
    # Groupby month, day, and hour to check outlier data  
    all_flight_data_group = all_flight_data.groupby(['Month','Day','Hour']).agg({'aircraft_type':'count'})
    all_flight_data_group = all_flight_data_group.reset_index()
    all_flight_data_group = all_flight_data_group.sort_values(by = ['Month','Day','Hour'])

    if all_flight_data_group['aircraft_type'].max() > all_flight_data_group.aircraft_type.quantile(0.9) * 1.35 and all_flight_data_group.aircraft_type.quantile(0.9) >=3: # and 0 in all_flight_data_group[all_flight_data_group['aircraft_type'] == all_flight_data_group['aircraft_type'].max()].Hour.to_list():
        print(file_path)
        outlier_hour_list.append(all_flight_data_group[all_flight_data_group['aircraft_type'] == all_flight_data_group['aircraft_type'].max()].Hour.to_list())
        print(all_flight_data_group[all_flight_data_group['aircraft_type'] == all_flight_data_group['aircraft_type'].max()].Month.to_list())
        print(all_flight_data_group[all_flight_data_group['aircraft_type'] == all_flight_data_group['aircraft_type'].max()].Day.to_list())
        print(all_flight_data_group[all_flight_data_group['aircraft_type'] == all_flight_data_group['aircraft_type'].max()].Hour.to_list())
        cnt_outlier += 1
        print(cnt)
    plt.plot(range(all_flight_data_group.shape[0]), all_flight_data_group.aircraft_type)  # Plot the chart
    plt.show()  # display  
    cnt += 1